## SWU Tic Tac Toe
โปรแกรมนี้เป็นเกมโอเอกซ์ สำหรับสองคนผ่านเครือข่ายโดยอาศัยโปรโตคอล MQTT (MQ Telemetry Transport)<br>
ผู้เล่นจะสื่อสารกันผ่าน MQTT Broker ซึ่งเป็น server กลางที่ช่วยให้ผู้เล่นที่อยู่คนละเครือข่ายสามารถสื่อสารกันได้ <br><br>
__ลำดับในการทำงาน__
1. เริ่มเกมด้วยการเป่ายิ้งฉุบ ผู้ชนะจะได้เครื่องหมาย X ผู้แพ้ได้เครื่องหมาย O โดยผู้ชนะจะได้เริ่มเกมก่อน
2. ผู้เล่นเลือกตำแหน่งวางเครื่องหมายในตาราง (board) และส่งข้อมูลตำแหน่งที่เลือกไปให้คู่ต่อสู้ผ่าน MQTT
3. รอจนกว่าจะได้ตำแหน่งที่เลือกจากคู่ต่อสู้  วางเครื่องหมายของคู่ต่อสู้เลือกในตำแหน่งที่ได้รับ
4. ตรวจสอบการแพ้,ชนะ หรือเสมอ หากรู้ผลแล้วจบการทำงาน
5. หากยังไม่มีการแพ้,ชนะ หรือเสมอ วนกลับไปทำในลำดับที่ 2 ใหม่

__อ้างอิง__: โปรแกรมนี้ดัดแปลงและปรับปรุงมาจาก
- mqtt: https://www.emqx.com/en/blog/how-to-use-mqtt-in-python
- mqtt: http://www.steves-internet-guide.com/receiving-messages-mqtt-python-client/
- rock-paper-scissors: https://realpython.com/python-rock-paper-scissors/
- Tic-Tac-Toe: https://geekflare.com/tic-tac-toe-python-code/

In [ ]:
#ติดตั้ง paho-mqtt library เพื่อใช้ในการสื่อสารผ่าน MQTT โปรโตคอล
#!pip install paho-mqtt

### Import library และสร้างตัวแปร

เรียกใช้งาน Library ต่างๆเพื่อให้สามารถพัฒนาโปรแกรมได้อย่างสะดวกและรวดเร็ว<br>
และประกาศตัวแปรโกลบอล(global variable)เพื่อให้สามารถเรียกใช้งานได้จากทุกจุดในโปรแกรม

In [ ]:
import random
import time
import json
from enum import IntEnum

#สำหรับการสร้าง Queue ในการเก็บข้อความจาก MQTT
from queue import Queue
#สำหรับการสื่อสาร MQTT
from paho.mqtt import client as mqtt_client

#เก็บค่าเครื่องหมายประจำตัวผู้เล่นมีค่าเป็น O หรือ X 
mysymbol=""

#กำหนดค่าตัวแปรต่างๆสำหรับ MQTT
debug = False
#ชื่อของ MQTT broker ที่ต้องการใช้
broker = "mqtt.eclipseprojects.io"
#TCP port สำหรับการสื่อสารใน MQTT 
port = 1883

#ชื่อ MQTT Topic ของผู้เล่น
my_topic = "your/topic"
#ชื่อ MQTT Topic ของคู่ต่อสู้
friend_topic = "friend/topic"

#จำนวน MQTT message ที่ได้รับ
msg_count = 0
#Queue ที่ใช้พักข้อมูลที่ส่งมาทาง MQTT topic ที่subscirbe ไว้
msg_q=Queue()

#สร้าง client ID with pub prefix randomly เพื่อใช้ในการเชื่อมต่อกับ MQTT Broker
client_id = f'python-mqtt-{random.randint(0, 1000)}'
# user และ password ที่จำเป็นต้องใช้ หากเชื่อมต่อกับ MQTT Broker ที่ไม่เป็นสาธารณะ
# username = 'emqx'
# password = 'public'

### MQTT
เตรียม Library และฟังก์ชั่น สำหรับการทำงานในส่วนที่เกี่ยวข้องกับ MQTT

In [ ]:
#ฟังก์ชั่นสำหรับเชื่อมต่อกับ mqtt broker
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    
    #สร้างตัวแปร cleint เพื่อใช้ในการเชื่อมต่อกับ mqtt broker
    client = mqtt_client.Client(client_id)
    #กำหนดค่า username และ password ที่ใช้ในการเชื่อมต่อ
    #client.username_pw_set(username, password)
    #กำหนด client เรียกใช้งานฟังก์ชั่น on_connect() ที่กำหนดไว้เมื่อการเชื่อมต่อกับ mqtt broker สำเร็จ
    client.on_connect = on_connect
    #ทำการเชื่อมต่อไปยัง mqtt broker และ หายเลข port ที่กำหนดไว้
    client.connect(broker, port)
    #คืนค่าตัวแปร client (client object) เพื่อใช้ในการสื่อสารกับ mqtt broker ต่อไป
    return client

#ฟังก์ชั่นสำหรับการส่ง message ไปยัง mqtt broker ใน topic ที่ได้ประกาศไว้
def publish(client, msg):
    result = client.publish(my_topic, msg)
    # ค่าของ result เป็นไปได้สองค่าคือ 0: ส่งข้อมูลได้สำเร็จ หรือ 1: ส่งข้อมูลไม่สำเร็จ
    status = result[0]
    if status == 0:
        if debug :
            print(f"Send `{msg}` to topic `{my_topic}`")
    else:
        print(f"Failed to send message to topic {my_topic}")

#ฟังก์ชั่นสำหรับจัดการกับข้อมูลที่เข้ามาทาง topic ที่ได้ subscribe ไว้
#โดยฟังก์ชั่นนี้จะถูกเรียกอัตโนมัติเมื่อมี message เข้ามา
#การทำงาน หากมีข้อมูลเข้ามาจะนำไปต่อไว้ใน queue เพื่อรอการนำไปใช้
def on_message(client, userdata, msg):
        if debug :
            print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        msg_q.put(msg) #add a message to the message queue
        
# ฟังก์ชั่นสำหรับ การสมัครรับฟังข้อมูลจาก topic ที่สนใจ
# ในที่นี้คือ topic ของคู่แข่ง
#โดยจะกำหนดฟังก์ชั่นที่จะช่วยในการจัดการกับ message ที่เข้ามาไว้ด้วย
def subscribe(client: mqtt_client):
    #สมัครรับข้อมูลจาก topic ของคู่ต่อสู้
    client.subscribe(friend_topic)
    #กำหนดให้เมื่อมี message เข้ามาให้ไปเรียก on_message ที่ประกาศไว้ด้านบนอัตโนมัติ
    client.on_message = on_message

#ฟังก์ชั่นสำหรับ การอ่าน message จาก queue ครั้งละ message
#คืนค่า message ที่อ่านมาได้ หากไม่มีข้อมูลจะคืนค่าเป็นข้อความที่ไม่มีข้อมูล
def get_msg():
    ret_msg=""
    if not msg_q.empty():
        message = msg_q.get()
        if message is not None:
             ret_msg=str(message.payload.decode("utf-8"))
    #คืนค่า message ไปให้
    return ret_msg

#ฟังก์ชั่นสำหรับอ่านค่า row และ col จาก mqtt message
#และคืนค่า row และ col 
def ttt_msg(in_msg):
    json_str = json.loads(in_msg)
    return json_str['row'], json_str['col']

### Rock Paper Scissors
เตรียม Library และฟังก์ชั่น สำหรับการทำงานในส่วนที่เกี่ยวข้องกับ Rock Paper Scissors

In [ ]:
#rock paper scissors
#สร้าง class ข้อมูลที่ใช้ในการเก็บผลเป่ายิ้งฉุบ
class RPSResult(IntEnum):
    #result status
    Lose = 0
    Win = 1
    Draw = 2
    
#สร้าง class ข้อมูลชนิดรูปแบบการเป่ายิ้งฉุบ
class Action(IntEnum):
    Rock = 0
    Paper = 1
    Scissors = 2

#ฟังก์ชั่นในการรับตัวเลือกเป่ายิ้งฉุบ
#คืนค่าที่ผู้เล่นเลือก คืนค่าเป็นแบบ Action class
def get_user_selection():
    user_input = input("Enter a choice (rock[0], paper[1], scissors[2]): ")
    while (user_input not in ['0','1','2']):
        user_input=input("Enter a choice (rock[0], paper[1], scissors[2]): ")

    selection = int(user_input)
    action = Action(selection)
    return action


#ฟังก์ชั่น หาผู้ชนะในการเป่ายิ้งฉุบ
#คืนค่าผลเป็น แพ้ ชนะ เสมอ ในแบบ RPSResult  class
def determine_winner(user_action, friend_action):
    result = RPSResult.Win
    if user_action == friend_action:
        print(f"Both players selected {user_action.name}. It's a tie!")
        result = RPSResult.Draw
    elif user_action == Action.Rock:
        if friend_action == Action.Scissors:
            print("Rock smashes scissors! You win!")
        else:
            print("Paper covers rock! You lose.")
            result = RPSResult.Lose
    elif user_action == Action.Paper:
        if friend_action == Action.Rock:
            print("Paper covers rock! You win!")
        else:
            print("Scissors cuts paper! You lose.")
            result = RPSResult.Lose
    elif user_action == Action.Scissors:
        if friend_action == Action.Paper:
            print("Scissors cuts paper! You win!")
        else:
            print("Rock smashes scissors! You lose.")
            result = RPSResult.Lose
    return result



### Tic Tac Toe 

เตรียมฟังก์ชั่น(เขียนในแบบ class) สำหรับการทำงานในส่วนที่เกี่ยวข้องกับ Tic Tac Toe

ตาราง __3 x 3__ การระบุตำแหน่งในตารางเป็นตัวเลข row, col <br> 
ในภาษา phthon ตำแหน่งแรกของสมาชิกจะเริ่มจาก 0 <br>
การอ้างอิงแบบคู่ลำดับ (row, col) เช่น (0,0) คือ A, (1,1) คือ E <br>
&emsp; &emsp;&emsp;&emsp; col <br>
&emsp; &emsp;&emsp;&ensp;0&ensp;1&ensp;2<br>
row 0 -> A  B  C <br>
row 1 -> D  E  F <br>
row 2 -> G  H  I <br>


In [ ]:
#Class TicTacToe 
#ทำหน้าที่สร้างตาราง (board) และเก็บข้อมูลการเล่น
class TicTacToe:
    
    #class constructor
    def __init__(self):
        self.board = []
    
    #ฟังก์ชั่นสร้างตาราง
    def create_board(self):
        for i in range(3):
            row = []
            for j in range(3):
                row.append(' ')
            self.board.append(row)
    
    #ฟังก์ชั่นนำข้อมูลลงในตารางตามตำแหน่งที่กำหนด
    def fix_spot(self, row, col, player):
        self.board[row][col] = player
        
    #ฟังก์ชั่นตรวจสอบว่าตำแหน่งที่เลือกว่างอยู่หรือไม่
    def is_spot_fixed(self, row, col):
        if (self.board[row][col] != ' '):
            return True
        return False
    
    #ฟังก์ชั่นตรวจสอบว่าชนะหรือยัง
    def is_player_win(self, player):
        win = None

        n = len(self.board)

        # checking rows
        for i in range(n):
            win = True
            for j in range(n):
                if self.board[i][j] != player:
                    win = False
                    break
            if win:
                return win

        # checking columns
        for i in range(n):
            win = True
            for j in range(n):
                if self.board[j][i] != player:
                    win = False
                    break
            if win:
                return win

        # checking diagonals
        win = True
        for i in range(n):
            if self.board[i][i] != player:
                win = False
                break
        if win:
            return win

        win = True
        for i in range(n):
            if self.board[i][n - 1 - i] != player:
                win = False
                break
        if win:
            return win
        return False

        for row in self.board:
            for item in row:
                if item == ' ':
                    return False
        return True

    #ฟังก์ชั่นตรวจสอบว่าช่องในตารางถูกใส่ค่าจนเต็มหรือยัง
    def is_board_filled(self):
        for row in self.board:
            for item in row:
                if item == ' ':
                    return False
        return True

    #ฟังก์ชั่นในการสลับเครื่องหมายของผู้เล่น
    def swap_player_turn(self, player):
        return 'X' if player == 'O' else 'O'

    #ฟังก์ชั่นแสดงตาราง พร้อมข้อมูลในตาราง
    def show_board(self):
        for i in range(len(self.board)):
            row = self.board[i]
            row_str=""
            for j in range(len(row)):
                row_str += '  ' if row[j] == ' ' else row[j]
                if j != len(row)-1:
                    row_str+='|'
            print(row_str)
            if i != len(self.board)-1:
                print("-+-+-")
        print()
        
    #ฟังก์ชั่นรับข้อมูลตำแหน่งที่ผู้เล่นเลือก พร้อมตรวจสอบว่าตำแหน่งที่เลือกว่างอยู่หรือไม่
    #หากไม่ว่าจะให้เลือกใหม่จนกว่าจะถูกต้อง
    #คืนค่าเป็นตำแหน่งที่ผู้เล่นเลือก
    def take_input(self):
        row=0
        col =0
        while 1:
            try:
                row, col = list( map(int, input("Enter row and column numbers to fix spot: ").split()))
                if (row in [1,2,3] and col in [1,2,3]):
                    break
            except:
                pass
            print("wrong input try again!")
        return row,col
            

# เริ่มส่วนการทำงานหลัก

### เชื่อมต่อกับ Broker
สร้าง Mqtt client เชื่อมต่อกับ Broker และค่อยฟังข้อมูลจาก Broker ในช่อง (Topic) ตามที่กำหนด

In [ ]:
#สร้างตัวแปร "client" (object) ซึ่งเป็นทำหน้าที่เป็น Mqtt Client ที่เชื่อมต่อกับ MQTT Broker
#และ client จะทำหน้าที่ในการรับส่งข้อมูล ( subscribe and publish) กับ Broker
client = connect_mqtt()

#loop_start() เป็นฟังก์ชั่นที่ทำให้ Mqtt client เริ่มทำงานเป็นวงรอบ (loop) เพื่อค่อยส่ง/รับข้อมูลกับ Broker
client.loop_start()

# client คอยรับของมูลจาก Topic ที่กำหนด
subscribe(client)

### Rock Paper Scissors
เป่ายิ้งฉุบ ให้ชนะได้เริ่มเล่นก่อน รับตัวเลือกและส่งค่าไปให้กับคู่ต่อสู้ผ่าน Mqtt

In [ ]:
#rock paper scissors

#กำหนดค่าเริ่มต้นของผลลัพธ์ให้เป็นเสมอ
RPS_status = RPSResult.Draw

#ทำการเป่ายิ้งฉุบจนกว่าจะมีผู้ชนะ หากเสมอเป่ายิ้งฉุบใหม่
#กำหนด while loop ทำวนไปจนกว่าผลจะไม่เป็นเสมอ
while RPS_status == RPSResult.Draw:
    #รับข้อมูลจากผู้เล่น
    user_action = get_user_selection()
    
    #แปลงข้อมูลให้อยู่ในรูปแบบ JSON และ Publish ข้อมูลไปให้คู่แข่ง
    #ตัวอย่างข้อมูล ->  {"rpm": 0}, {"rpm": 1}, หรือ {"rpm": 2}
    rps_msg =json.dumps({"rps":user_action.value})
    publish(client, rps_msg)
    
    #ตรวจสอบว่ามีข้อมูลมาใน Mqtt Topic ที่ subscribe ไว้หรือไม่
    #ดึงข้อมูลจาก Queue ที่เราได้ไสร้างไว้เพื่อเก็บข้อมูลที่มาจาก Broker
    in_msg = get_msg()
    #หากไม่มีข้อมูล หรือ ข้อมูลไม่มีค่าเกี่ยวกับ rock paper scissors (rps) ให้ดึงข้อมูลมาใหม่จนกว่า
    #จะเป็นข้อมูล rps 
    while (len(in_msg) == 0) or ("rps" not in json.loads(in_msg)):
        time.sleep(3)
        in_msg = get_msg()
    
    #ดึงค่าตัวเลือกของคู่ต่อสู้ และแปลงค่าให้เป็นตัวเลข
    selection = int(json.loads(in_msg)["rps"])
    #แปลงค่าตัวเลือก 0,1,2 ให้เป็นขนิด Action
    opponent_action = Action(selection)
    #เปรียบเทียบเพื่อหาผู้ชนะ เก็บผลลัพธ์ไว้ที่ RPS_status และนำไปใช้ในการตัดสินใจใน while loop 
    RPS_status=determine_winner(user_action,opponent_action)

#----- สิ้นสุด rock paper scissors --------------------------------


### Tic Tac Toe 


In [ ]:
#----- เริ่ม Tic Tac Toe ---------------------------------------------
#สร้างตัวแปร (object) tic_tac_toe เพื่อใช้ในการแข่ง
tic_tac_toe = TicTacToe()
#สร้างตาราง (board)
tic_tac_toe.create_board()
#สร้างตัวแปร player เพื่อไว้เก็บว่าจะได้เป็น O หรือ X
player=""


#หากผู้เล่นเป็นผู้ชนะการเป่ายิ้งฉุบ จะได้เป็น X และได้เริ่มเล่นก่อน
#หากแพ้ จะได้เป็น O และได้เริ่มทีหลัง
if (RPS_status == RPSResult.Win):
    mysymbol=player = 'X'
    print("Your are "+player+" player")
    
    #แสดง board 
    tic_tac_toe.show_board()
    
    #รับตำแหน่งที่ผู้ใช้ต้องการวางเครื่องหมาย
    # while 1 คือ Infinite loop เพื่อวนรับข้อมูลจากผู้เล่นจนกว่าจะถูกต้อง
    while 1:
        #รับข้อมูลจากผู้เล่น
        row, col = tic_tac_toe.take_input()
        #แสดงบรรทัดว่าง
        print()
        #ตรวจดูว่าเลือกช่องที่มีการใส่เครื่องหมายแล้วหรือเปล่า
        #หากใช่ก็จะแสดงข้อความและวนไปรับข้อมูลใหม่
        #หากเป็นช่องที่ว่างอยู่ก็จะออกจาก loop 
        if not(tic_tac_toe.is_spot_fixed(row-1, col-1)):
            # break ใช้ในการหยุด while loop (ออกจาก loop)
            break
        #แสดงข้อควาให้ใส่ค่าใหม่ในกรณีที่ใส่ไม่ถูกต้อง
        print("The selected spot is fixed. Try another spot.")
    
    #ใส่เครื่องหมายในช่องที่กำหนด
    tic_tac_toe.fix_spot(row - 1, col - 1, player)
    #ส่งตำแหน่งที่เลือกไปให้คู่ต่อสู้
    #โดยข้อมูลที่ส่งอยู่ในรูปแบบ JSON -> {'row':0, 'col':0}, {'row':0, 'col':1}, หรืออื่นๆ
    #row มีค่าอยู่ระหว่าง 0 ถึง 2 และ col มีค่าอยู่ระหว่าง 0 ถึง 2 เช่นกัน
    publish_str = json.dumps({'row':row,'col':col})
    publish(client, publish_str)
else:
    #ในกรณีที่เป่ายิ้งฉุบแพ้ ก็จะได้เครื่องหมาย O และได้เล่นทีหลัง
    mysymbol=player = 'O'
    print("Your are "+player+" player")
    
#แสดง board หากเป็นผู้ชนะจะเห็นเครื่องหมายในตำแหน่งที่เลือกไว้
#หากเป่ายิ้งฉุบแพ้ก็จะเห็นเป็น board ว่างๆ
tic_tac_toe.show_board()

# while 1 คือ Infinite loop ผลัดกันใส่ข้อมูลจนกว่าจะได้ผล แพ้ ชนะ หรือเสมอ
while 1:
    #ตรวจข้อมูลที่ได้รับ มาทาง Mqtt topic ที่ลงทะเบียนไว้
    in_msg = get_msg()
    while (len(in_msg) == 0) or ("row" not in json.loads(in_msg)):
        #หากไม่มีข้อมูล หรือข้อมูลไม่เกี่ยวกับตำแหน่งในตารางก็จะวนตรวจข้อมูลจะกว่าจะใช่
        time.sleep(3)
        in_msg = get_msg()
        
    #หากมีข้อมูลเกี่ยวกับตำแหน่งมาจริง แสดงว่าคู่ต่อสู้เลือกในแหน่งในตา(turn)ของตัวเองแล้ว
    if (len(in_msg)>0):
        # สลับตา(trun) เป็นของผู้เล่น (เปลี่ยนเครื่องหมาย) เป็นของคู่ต่อสู้
        #เนื่องจากข้อมูลที่ได้เป็นตำแหน่งที่คู่ต่อสู้เลือก
        player = tic_tac_toe.swap_player_turn(player)
        #สกัดตำแหน่องที่ต้องการวางเครื่องหมาย จากข้อความ
        row,col = ttt_msg(in_msg)
        #ใส่เครื่องหมายในตาราง
        tic_tac_toe.fix_spot(row - 1, col - 1, player)
        #แสดง board
        tic_tac_toe.show_board()
        
        #ตรวจดูว่ามีผู้ชนะหรือยัง หากมีแล้วจะออกจาก Infinite loop (while 1)
        if tic_tac_toe.is_player_win(player):
            if (player == mysymbol):
                print('You win !!!  *( ^ o ^)* ')
            else:
                 print('You lose !!!  (; _ ;)')
            break

        # ตรวจดูว่าเสมอหรือไม่ หากใช่จะออกจาก Infinite loop (while 1)
        if tic_tac_toe.is_board_filled():
            print("Match Draw!")
            break

        # เปลี่ยนตา (turn) เป็นของผู้เล่น (เปลี่ยนเครื่องหมาย)
        player = tic_tac_toe.swap_player_turn(player)

        # รับตำแหน่งจากผู้เล่น
        while 1:
            row, col = tic_tac_toe.take_input()
            print()
            if not(tic_tac_toe.is_spot_fixed(row-1, col-1)):
                break
            print("The selected spot is fixed. Try another spot.")
        
        # ใส่เครื่องหมายในตำแหน่งที่เลือก และแสดง board
        tic_tac_toe.fix_spot(row - 1, col - 1, player)
        tic_tac_toe.show_board()
        # ส่งข้อมูลให้คู่ต่อสู้
        publish_str = json.dumps({'row':row,'col':col})
        publish(client, publish_str)
        
        #ตรวจดูว่ามีผู้ชนะหรือยัง หากมีแล้วจะออกจาก Infinite loop (while 1)
        if tic_tac_toe.is_player_win(player):
            if (player == mysymbol):
                print('You win !!!  *( ^ o ^)* ')
            else:
                 print('You lose !!!  (; _ ;)')
            break

        # ตรวจดูว่าเสมอหรือไม่ หากใช่จะออกจาก Infinite loop (while 1)
        if tic_tac_toe.is_board_filled():
            print("Match Draw!")
            break


### จบการทำงาน

In [ ]:
#ยกเลิกการเชื่อมต่อกับ Mqtt broker
client.loop_stop()
client.disconnect()